<a href="https://colab.research.google.com/github/fpiedra47/regresion-logistica/blob/main/wine_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsOneClassifier

In [ ]:
col_names = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins','Color intensity', 'Hue', 'OD280/OD315 of diluted wines','Proline']
### Figure this out!
#df = pd.read_csv("/content/drive/My Drive/CodingDojo/Codigos/week7/wine.data")
df = pd.read_csv("/content/drive/My Drive/CodingDojo/Codigos/week7/wine.data", header = None,names = col_names)

In [ ]:
df.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
# Imprimir cuántas clasescopy
print('Class labels', np.unique(df['Class label']))

Class labels [1 2 3]


In [ ]:
# Las clases no están equilibradas
df['Class label'].value_counts(dropna = False)

2    71
1    59
3    48
Name: Class label, dtype: int64

In [ ]:
# Organizar los datos en la matriz de características y el vector de destino
X = df.loc[:, df.columns[(df.columns != 'Class label')]]
y = df.loc[:, 'Class label'].values

In [ ]:
# En encuestas estadísticas,
# cuando las subpoblaciones dentro de una población general varían,
# Podría ser ventajoso muestrear cada subpoblación (estrato) de forma independiente
# La estratificación es el proceso de dividir a los miembros de la población en subgrupos homogéneos antes
# del muestreo.
#help(train_test_split)

In [ ]:
# Dividir en conjuntos de prueba y entrenamiento
# Proporcionar la matriz de etiquetas de clase y como argumento para estratificar asegura a amboscopy
# el conjunto de entrenamiento y los conjuntos de datos de prueba tienen las mismas proporciones de clase que el
# conjunto de datos original
X_train, X_test, y_train, y_test =train_test_split(X,
                                                   y,
                                                   test_size=0.3, 
                                                   random_state=0, 
                                                   stratify=y)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{1: 41, 2: 50, 3: 33}

In [ ]:
# Estandarizar datos
scaler = StandardScaler()
# Ajustar solo al conjunto de entrenamiento
scaler.fit(X_train)
# Aplicar  transform tanto al conjunto de entrenamiento como al conjunto de prueba
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
"Tanto la precisión de entrenamiento como de prueba (ambas al 100 por ciento) indican que nuestro modelo hace un trabajo perfecto en ambos conjuntos de datos. Cuando se accede a los términos de intercepción a través del atributo `log_reg.intercept_`, vemos que la matriz devuelve tres valores."

In [ ]:
log_reg = LogisticRegression(penalty='l1',
                        C=1.0,
                        solver='liblinear',
                        multi_class='ovr')
log_reg.fit(X_train, y_train)
print('Training accuracy:', log_reg.score(X_train, y_train))
print('Test accuracy:', log_reg.score(X_test, y_test))

Training accuracy: 1.0
Test accuracy: 1.0


In [ ]:
log_reg.intercept_

array([-1.2636524 , -1.21604379, -2.37032134])

In [ ]:
log_reg.coef_
"La matriz de peso a la que se accede mediante el atributo log_reg.coef_, que contiene tres filas de coeficientes de peso, un vector de peso para cada clase. Cada fila consta de 13 pesos, donde cada peso se multiplica por la característica respectiva en el conjunto de datos de vino de 13 dimensiones para calcular la entrada neta."

array([[ 1.24595344,  0.18063786,  0.74505853, -1.16286989,  0.        ,
         0.        ,  1.16438326,  0.        ,  0.        ,  0.        ,
         0.        ,  0.55264744,  2.50961236],
       [-1.53687925, -0.38728738, -0.99494235,  0.36463375, -0.05965903,
         0.        ,  0.66788463,  0.        ,  0.        , -1.93414526,
         1.23373071,  0.        , -2.23237163],
       [ 0.13561808,  0.16858755,  0.35731326,  0.        ,  0.        ,
         0.        , -2.43748347,  0.        ,  0.        ,  1.56348234,
        -0.81879995, -0.49266694,  0.        ]])

In [ ]:
# La primera clase es la puntuación más alta, por lo que será la predicción 0 (la primera clase) para estos datos
log_reg.predict_proba(X_test[0:1])

array([[9.76605648e-01, 4.47689726e-04, 2.29466620e-02]])

In [ ]:
# También podemos obtener la clase asociada con la probabilidad más alta
log_reg.predict(X_test[0:1])

array([1])

**1 vs 1**

In [27]:
X = df.drop(columns = 'Class label')
y= df['Class label']

In [46]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [28]:
sc = StandardScaler()
sc.fit(X_train)

StandardScaler()

In [36]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
lg = LogisticRegression()
ovo = OneVsOneClassifier(lg)
ovo.fit(X_train, y_train)

OneVsOneClassifier(estimator=LogisticRegression())

In [40]:
print(ovo.score(X_train, y_train))

1.0


In [41]:
print(ovo.score(X_test, y_test))

1.0


**1 vs todos**

In [42]:
lg = LogisticRegression(multi_class = 'ovr')
lg.fit(X_train, y_train)
print(lg.score(X_train, y_train))

1.0


**KNN**

In [43]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print(knn.score(X_train, y_train))

0.967741935483871


**Bagged tree**

In [44]:
bt = BaggingClassifier()
bt.fit(X_train, y_train)
print(knn.score(X_train, y_train))

0.967741935483871


**Random Forest**

In [45]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train))

1.0


**¿Qué modelo funcionó mejor?**
Todos los modelos con fines educativos han predecido 1 o cerca del mismo valor. Por lo tanto, sería díficil cual es el mejor.
**¿Cuáles son las características más importantes de tus modelos? ¿Hay algún modelo que te haya gustado más y por qué?**
KNN me gusta por lo que son pocas líneas de código y predice bastante bien.